In [1]:
import numpy as np
import numpy.linalg as LA

In [2]:
srho = 0.25**2
spsi = 0.0436**2
alpha = 0.05

## Questão 1

In [3]:
def lp2lc(coords) -> np.array:
    rho, psi = coords
    return np.array([
        [rho*np.cos(psi)],
        [rho*np.sin(psi)],
    ])

def Q_lp2lc(obs) -> np.array:
    [rho, psi] = obs
    return np.array([
        [rho**2 * spsi * np.sin(psi)**2 + srho * np.cos(psi)**2, -0.5*(rho**2 * spsi+srho)*np.sin(2*psi)],
        [-0.5*(rho**2 * spsi+srho)*np.sin(2*psi), rho**2 * spsi * np.cos(psi)**2 + srho * np.sin(psi)**2]
    ])

## Questão 2

In [4]:
def R_lc2g(theta: float) -> np.array:
    return np.array([
            [ np.cos(theta),-np.sin(theta)],
            [ np.sin(theta), np.cos(theta)]
    ])

def A_lc2g(robot: np.array) -> np.array:
    x, y, theta = robot
    return np.array([
        [ np.cos(theta), -np.sin(theta), x],
        [ np.sin(theta), np.cos(theta), y],
        [0, 0, 1]
    ])

def lc2g(coords_lc: np.array, robot: np.array) -> np.array:
    return (A_lc2g(robot) @ np.array([[coords_lc[0]], [coords_lc[1]], [1]]))[:2]

def Q_lc2g(theta: float, Q_lc: np.array) -> np.array:
    return R_lc2g(theta) @ Q_lc @ R_lc2g(theta).T

## Questão 3

In [5]:
def g2lc(coords_g: np.array, robot: np.array) -> np.array:
    A_g2lc = LA.inv(A_lc2g(robot))
    gx, gy = coords_g
    return (A_g2lc @ np.array([[gx], [gy], [1]]))[:2]

def C(theta):
    return R_lc2g(theta).T

def lc2lp(coords_lc: np.array) -> np.array:
    l, r = coords_lc
    return np.array([
        [np.sqrt(l**2+r**2)],
        [np.arctan2(r,l)]
    ])

## Questão 4

In [6]:
def check_sensor(x: float, y: float, theta_robot: float = np.pi/6) -> bool:
    print(f'X: {x}   Y: {y}')
    D_MAX = 3
    ANG_MAX = np.pi/3
    x_robot = 1
    y_robot = 2
    l, r = g2lc([x,y], [x_robot, y_robot, theta_robot])
    rho, psi = lc2lp([l,r])
    print(rho,psi)
    
    if rho <= D_MAX and (-ANG_MAX <= psi) and (psi <= ANG_MAX):
        print(True,'\n')
    else:
        print(False,'\n')

In [7]:
check_sensor(3,4)

X: 3   Y: 4
[[2.82842712]] [[0.26179939]]
True 



## Questão 5

In [8]:
z1 = [3, 0]
z2 = [4, -0.646] 
z3 = [np.sqrt(2), np.pi/8]
Z = [z1,z2,z3]
Z = np.array(Z)
for z in Z:
    print(lp2lc(z))

[[3.]
 [0.]]
[[ 3.19399268]
 [-2.40798895]]
[[1.30656296]
 [0.5411961 ]]


In [9]:
data = np.genfromtxt('./data/valoresEP2.csv', delimiter=',')

In [10]:
observations = data[:,2:]

In [14]:
def Lj(land, obs, robot_t):
    '''
    land = [mu, sigma]
    mu = [x_bar, y_bar]
    obs = [rho, psi]
    '''
    return C(robot_t[2]) @ land['sigma'] @ C(robot_t[2]).T + Q_lp2lc(obs)
    
def W(observations_t, landmarks, robot_t):
    '''
    Observations = todas as obs de um mesmo timestamp
    '''
    m = len(landmarks)
    n = len(observations_t)
    W = np.empty((n,m))
    for i, obs in enumerate(observations_t):
#         p_obs = np.empty(m)
        for j, land in enumerate(landmarks):
            d = lp2lc(obs) - g2lc(land['mu'], robot_t)
            L = Lj(land, obs, robot_t)
            W[i][j] = 1 / np.sqrt(2*np.pi*LA.det(L)) * np.exp(-(d.T @ LA.inv(L) @ d)/2)
        
    return W

$$M=\left\{\left\{\left[\begin{array}{l}
2 \\
1
\end{array}\right],\left[\begin{array}{cc}
1 / 9 & 0 \\
0 & 1
\end{array}\right]\right\}\right\},\left\{\left\{\left[\begin{array}{c}
3 \\
-1
\end{array}\right],\left[\begin{array}{ll}
5 / 9 & 4 / 9 \\
5 / 9 & 4 / 8
\end{array}\right]\right\},\left\{\left[\begin{array}{c}
5 / 2 \\
2
\end{array}\right],\left[\begin{array}{cc}
1 / 9 & 0 \\
0 & 1
\end{array}\right]\right\}\right\}$$

In [12]:
robot_t = np.array([4, 3, np.pi/2])
landmarks = [
    [[2,1], [[1/9,0],
             [0,1.]]],
    [[3,-1], [[5/9,4/9],
              [5/9,4/8.]]],
    [[5/2,2], [[1/9,0],
             [0,1.]]],    
]
landmarks = [{'mu':np.array(l[0]), 'sigma':np.array(l[1])} for l in landmarks]

In [15]:
W(Z, landmarks, robot_t)

array([[1.41318828e-12, 5.40098041e-63, 8.98021412e-08],
       [4.64536733e-37, 9.71899186e-27, 2.47978685e-28],
       [2.53383637e-06, 7.50504663e-68, 3.12119160e-03]])